In [ ]:
import pandas as pd


In [ ]:
recipes = pd.read_csv('/content/drive/MyDrive/Iron Hack/Final project/RAW_recipes.csv')

In [ ]:
recipes = recipes.drop(['id', 'contributor_id', 'submitted', 'nutrition', 'description'], axis=1)

In [ ]:
steps = recipes['steps'].astype(str).tolist()

In [ ]:
import spacy
spacy.cli.download("en_core_web_md")


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
nlp = spacy.load('en_core_web_md')

def embed_text(text):
    doc = nlp(text)
    return doc.vector

embeddings = [embed_text(step) for step in steps]


In [ ]:
import numpy as np

embedding_matrix = np.array(embeddings).astype('float32')

np.save('embedding_matrix.npy', embedding_matrix)

In [ ]:
%pip install faiss-gpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 10.8 MB/s eta 0:00:00


In [ ]:
embedding_matrix = np.load('embedding_matrix.npy')

In [ ]:
import faiss
import numpy as np

faiss.normalize_L2(embedding_matrix)


index = faiss.IndexFlatIP(embedding_matrix.shape[1])
index.add(embedding_matrix)


In [ ]:
def semantic_search(query, df, index, k=5):
    query_embedding = embed_text(query)

    query_embedding = np.expand_dims(query_embedding, axis=0)

    faiss.normalize_L2(query_embedding)

    _, result_indices = index.search(query_embedding, k)


    print(f"Results for query: {query}\n")
    print("Entire Dataset:")
    for i, idx in enumerate(result_indices.flatten()):
        print(f"Rank {i+1} - Item {idx + 1}:")
        print(df.iloc[idx])
        print("\n")

query = "Healthy salad with quinoa and vegetables"
semantic_search(query, recipes, index)

Results for query: Healthy salad with quinoa and vegetables

Entire Dataset:
Rank 1 - Item 165655:
name                    prawn  shrimp  cocktail with dill dressing
minutes                                                         30
tags             ['30-minutes-or-less', 'time-to-make', 'course...
n_steps                                                          2
steps            ['for the dill dressing - combine the egg mayo...
ingredients      ['mayonnaise', 'dijon mustard', 'dill', 'water...
n_ingredients                                                   10
Name: 165654, dtype: object


Rank 2 - Item 140837:
name                                          mushroom pasta for 2
minutes                                                         35
tags             ['60-minutes-or-less', 'time-to-make', 'course...
n_steps                                                          2
steps            ['in a skillet heat oil and add the onion& gar...
ingredients      ['onions', 'garlic', 'lite o

In [ ]:
import pickle
import faiss
import numpy as np

with open('faiss_index.pkl', 'wb') as index_file:
    pickle.dump(index, index_file)

np.save('embedding_matrix.npy', embedding_matrix)


In [ ]:
from google.colab import files

files.download('faiss_index.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>